In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using RigidBodyTreeInspector
using DrakeVisualizer
using Plots

using BilevelTrajOpt

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/9dd84cf97c4fc16956f39829c81f3b6907c8fc83-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
env = parse_contacts(mechanism, urdf, .5);
x0 = MechanismState(mechanism)
Δt = 0.01
sim_data = get_sim_data(x0,env,Δt,true);

In [3]:
# q0 = [1., 0., 0., 0., 0., 0., 0.]
# v0 = [0., 0., 0., .25, 0., 0.]
# q0 = [1.0, 1.02527e-44, -3.30872e-28, -8.9132e-46, 0.016266, 1.5112e-19, -1.40034e-9]
# v0 = [-9.0404e-42, -2.86986e-41, -3.59817e-44, 0.1266, 1.51121e-17, -0.439901]
# q0 = [1.0, 1.9873e-14, -2.33452e-10, 4.82734e-19, 0.0174996, 8.11268e-7, -3.72712e-7]
# v0 = [9.9293e-17, -1.16731e-12, -2.65886e-20, 0.249991, 2.12753e-5, -0.441238]
q0 = [1.0, 1.02527e-44, -3.30872e-28, -8.9132e-46, 0.016266, 1.5112e-19, -1.40034e-9]
v0 = [-9.0404e-42, -2.86986e-41, -3.59817e-44, 0.1266, 1.51121e-17, -0.439901]

u0 = zeros(sim_data.num_v)
# set_configuration!(x0, q0)
# set_velocity!(x0, v0)
# setdirty!(x0)
# traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,implicit_contact=false)
# qnext = traj[1:sim_data.num_q,2]
# vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
# qnext = [1.0, 1.8722e-14, -2.33457e-10, 1.01223e-16, 0.0199995, 1.78903e-6, -9.99477e-9]
# vnext = [9.35363e-17, -1.16731e-12, 3.62294e-22, 0.249991, 9.77758e-5, 3.62716e-5]
qnext = [1.0, -3.49493e-44, -3.30872e-28, -1.0679e-45, 0.016266, 1.51127e-19, -4.08825e-9]
vnext = [-9.0404e-42, -2.86986e-41, -3.53162e-44, 7.50696e-9, 6.42133e-22, -2.68792e-7]

z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))
# z0 = [2.13309e-5, 4.89487e-5, 0.500026, 3.63767e-5, 0.450951, 1.00001]

num_steps_default = 5
α_vect_default = [1.^i for i = 1:num_steps_default]
c_vect_default = [0.01*(1.^i) for i = 1:num_steps_default]
I_vect_default = 1e-12*ones(num_steps_default)

τ_ip, x_sol_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=true);
τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=α_vect_default,c_vect=c_vect_default,I_vect=I_vect_default);
# τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false);

# z0 = zeros(sim_data.num_contacts*(sim_data.β_dim+2))
# N = 5
# α_vect=[1.^i for i = 1:N]
# c_vect=[10000.+50.^i for i = 1:N]
# I_vect=1e-10*ones(N)
# τ_auglag, x_sol_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,qnext,vnext,ip_method=false,α_vect=α_vect,c_vect=c_vect,I_vect=I_vect);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Solve_Succeeded


LoadError: [91mMethodError: no method matching string(::Expr)
The applicable method may be too new: running in world age 2757, while current world is 22016.[0m
Closest candidates are:
  string(::Any...) at strings/io.jl:120 (method too new to be called from this world context.)
  string([91m::BigInt[39m) at gmp.jl:568 (method too new to be called from this world context.)
  string([91m::BigFloat[39m) at mpfr.jl:885 (method too new to be called from this world context.)
  ...[39m

In [ ]:
display([0.38959, 0.226423, 1.68011, 0.226423, 6.97269e-9, 5.48421])
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)
# display("Explicit")
# display(traj[sim_data.num_q+sim_data.num_v+1:end,2])

In [ ]:
[0.38959, 0.226423, 1.68011, 0.226423, 6.97269e-9, 5.48421]

In [ ]:
display("IP")
display(τ_ip)
display("Aug Lag")
display(τ_auglag)

In [ ]:
BilevelTrajOpt.τ_total(traj[sim_data.num_q+sim_data.num_v+2:end,2],sim_data)

In [ ]:
ForwardDiff.jacobian(x̃ -> solve_implicit_contact_τ(sim_data,q0,v0,u0,z0,x̃[1:sim_data.num_q],x̃[sim_data.num_q+1:sim_data.num_q+sim_data.num_v];ip_method=false)[1],vcat(qnext,vnext))

In [ ]:
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1));
geometries = visual_elements(mechanism, URDFVisuals(urdf))
vis = Visualizer(mechanism, geometries);

In [ ]:
N = size(traj,2)
qs = [traj[1:num_positions(x0),i] for i in 1:N]
ts = cumsum([Δt for i in 1:N]);
RigidBodyTreeInspector.animate(vis, mechanism, ts, qs; realtimerate = .25);